In [ ]:
import json
import os
import shutil
from pathlib import Path

def prepare_dataset(images_source_dir, output_dir, json_paths):
    for split in ["train", "val", "test"]:
        os.makedirs(os.path.join(output_dir, split, "images"), exist_ok=True)
        os.makedirs(os.path.join(output_dir, split, "labels"), exist_ok=True)

    for split, json_path in json_paths.items():
        with open(json_path, 'r') as f:
            data = json.load(f)

        # Словари для удобного доступа
        categories = {cat["id"]: idx for idx, cat in enumerate(data["categories"])}
        images_info = {img["id"]: img for img in data["images"]}

        # Группируем аннотации по image_id
        from collections import defaultdict
        annotations_per_image = defaultdict(list)
        for ann in data["annotations"]:
            annotations_per_image[ann["image_id"]].append(ann)

        # Обрабатываем каждое изображение
        for img_info in data["images"]:
            image_id = img_info["id"]
            file_name = img_info["file_name"]
            src_img_path = os.path.join(images_source_dir, file_name)
            dst_img_path = os.path.join(output_dir, split, "images", file_name)

            # Копируем изображение
            shutil.copy(src_img_path, dst_img_path)

            # Создаем аннотацию
            label_file = Path(file_name).stem + ".txt"
            label_path = os.path.join(output_dir, split, "labels", label_file)

            # Проверяем аннотации для этого изображения
            annotations = annotations_per_image.get(image_id, [])
            has_valid_bbox = False

            with open(label_path, 'w') as f:  # Перезаписываем файл
                for ann in annotations:
                    if "bbox" in ann:
                        # Конвертируем bbox в YOLO-формат
                        bbox = ann["bbox"]
                        width, height = img_info["width"], img_info["height"]
                        x_center = (bbox[0] + bbox[2] / 2) / width
                        y_center = (bbox[1] + bbox[3] / 2) / height
                        norm_width = bbox[2] / width
                        norm_height = bbox[3] / height

                        # Записываем строку аннотации
                        class_id = categories[ann["category_id"]]
                        f.write(f"{class_id} {x_center:.6f} {y_center:.6f} {norm_width:.6f} {norm_height:.6f}\n")
                        has_valid_bbox = True

                # Если нет валидных bbox, оставляем файл пустым (negative sample)
                if not has_valid_bbox:
                    pass  # Файл уже создан пустым

images_source_dir = "eccv_18_all_images_sm/"
output_dir = "dataset/"
json_paths = {
    "train": "data/eccv_18_annotation_file/train_annotations.json",
    "val": "data/eccv_18_annotation_file/trans_val_annotations.json",
    "test": "data/eccv_18_annotation_file/trans_test_annotations.json"
}

prepare_dataset(images_source_dir, output_dir, json_paths)

In [10]:
def leave_only_first_line(directory):
    """
    Оставляет только первую строку в каждом .txt-файле указанной директории.
    """
    for filename in os.listdir(directory):
        if filename.endswith(".txt"):
            filepath = os.path.join(directory, filename)
            
            # Читаем все строки файла
            with open(filepath, 'r') as f:
                lines = f.readlines()
            
            # Если строк больше одной - перезаписываем файл с первой строкой
            if len(lines) > 1:
                with open(filepath, 'w') as f:
                    f.write(lines[0])

leave_only_first_line("dataset/train/labels")
leave_only_first_line("dataset/val/labels")
leave_only_first_line("dataset/test/labels")

In [11]:
class_mapping = {
    6: 0,
    1: 1,
    30: 2,
    9: 3,
    3: 4,
    11: 5,
    8: 6,
    16: 7,
    5: 8,
    10: 9,
    7: 10,
    99: 11,
    21: 12,
    34: 13,
    33: 14,
    51: 15
}

def process_labels(directory):
    """
    Обрабатывает все txt-файлы в директории, заменяя первый номер класса согласно маппингу.
    """
    for filename in os.listdir(directory):
        if filename.endswith(".txt"):
            filepath = os.path.join(directory, filename)
            
            with open(filepath, 'r') as f:
                lines = f.readlines()
            
            new_lines = []
            for line in lines:
                parts = line.strip().split()
                if len(parts) == 5:  # Проверяем, что строка содержит все 5 значений
                    old_class = int(parts[0])
                    if old_class in class_mapping:
                        # Заменяем только первый элемент, остальные оставляем как есть
                        new_line = f"{class_mapping[old_class]} {' '.join(parts[1:])}\n"
                        new_lines.append(new_line)
                    else:
                        print(f"Внимание: класс {old_class} не найден в маппинге (файл: {filename})")
                        new_lines.append(line)  # Оставляем строку без изменений
                else:
                    new_lines.append(line)  # Оставляем строки с неправильным форматом без изменений
            
            # Перезаписываем файл с обновленными данными
            with open(filepath, 'w') as f:
                f.writelines(new_lines)




In [ ]:
process_labels("dataset/train/labels")
process_labels("dataset/val/labels")
process_labels("dataset/test/labels")

In [1]:
from ultralytics import YOLO
import yaml

# Загружаем конфиг
with open("args.yaml", "r") as f:
    args = yaml.safe_load(f)


In [ ]:
model = YOLO(args["model"])
model.train(**args)